In [639]:
import chess
from IPython.display import display, clear_output
from time import sleep
from collections import defaultdict, Counter
import random
from queue import Queue

In [641]:
def is_move_losing(board, move):
    '''It check's if after the move the opponent has any move (it's stalmate) or if the opponent can capture queen'''
    board.push_uci(str(move))
    return not board.legal_moves or any(True for m in board.legal_moves if m is not None and str(m).endswith(str(move)[-2:]))
    
def is_move_winning(board, move):
    '''Simple checking if the move is winning the game'''
    board.push(move)
    try:
        outcome = board.outcome().winner
    except Exception:
        outcome = False
    return outcome

def is_white_king_near_edge(board):
    '''As the name says it's checking if the black king is on the edge of the board'''
    king_square = chess.square_name(board.king(chess.WHITE))
    return any(edge_names in king_square for edge_names in ['a','b', 'g','h', '1','2','7', '8'])

def calculate_black_king_distance(board):
    '''It calculates black king distance from center of the board
       We are using Euclidean distance becuase in our situation it's better than Chebyshev distance'''
    row = board.king(chess.BLACK) // 8
    column = board.king(chess.BLACK) % 8 
    return ((3.5 - row)**2 + (3.5 - column)**2)**0.5

def find_best_queen_move(board, move):
    start_distance = calculate_black_king_distance(board)
    quality_of_move = 1
    board.push_uci(str(move))
    for move in list(board.legal_moves):
        board.push_uci(str(move))
        if calculate_black_king_distance(board) < start_distance:
            quality_of_move = -1
            board.pop()
            break
        if calculate_black_king_distance(board) == start_distance:
            quality_of_move = 0
        board.pop()
    board.pop()
    return quality_of_move


In [642]:
def find_best_king_move(board: chess.Board, target_square, banned_squares = set()):
    '''https://en.wikipedia.org/wiki/Breadth-first_search'''
    explored = set()
    start_square = board.king(chess.WHITE)
    target_square = chess.parse_square(target_square)
    parents = {}
    q = Queue()
    q.put(start_square)
    explored.add(start_square)
    while not q.empty():
        current_square = q.get()
        board.set_piece_at(board.king(chess.WHITE),None)
        board.set_piece_at(current_square, chess.Piece(chess.KING, chess.WHITE))
        for move in list(board.legal_moves):
            if board.piece_at(chess.parse_square(str(move)[:2])).symbol() == "Q" or is_move_losing(board.copy(), move):
                continue
            if move.to_square not in explored and move.to_square not in banned_squares:
                explored.add(move.to_square)
                parents[move.to_square] = current_square
                q.put(move.to_square)
                if move.to_square == target_square:
                    break
    while True:
        if target_square not in parents:
            return None
        if parents[target_square] == start_square:
            return chess.Move(start_square, target_square)
        target_square = parents[target_square]
    
def black_king_only_one_square(board):
    if board.turn == chess.WHITE:
        board.push_uci("0000")
    return board.legal_moves.count() == 1

In [643]:
def is_black_king_in_corner(board):
    king_square = str(chess.square_name(board.king(chess.BLACK)))
    return king_square in ['a1','b1','a2','a8','a7','b8','h1','h2','g1','h8','h7','g8']
def get_king_final_position(board):
    king_square = chess.square_name(board.king(chess.BLACK))
    if king_square in ["a1", "b1", "a2"]:
        return "c3"
    if king_square in ["a8", "b8", "a7"]:
       return "c6" 
    if king_square in ["h1", "g1", "h2"]:
        return "f3"
    if king_square in ["h8", "g8", "h7"]:
        return "f6"
    return None
def calculate_best_move(board):
    candidate_moves = defaultdict(lambda: [])
    best_king_move = None
    if is_black_king_in_corner(board.copy()) and black_king_only_one_square(board.copy()):
        best_king_move = find_best_king_move(board.copy(), get_king_final_position(board.copy()))
    for move in board.legal_moves:
        if is_move_winning(board.copy(), move):
            return move
        if is_move_losing(board.copy(), move):
            continue
        if move == best_king_move:
            return move
        if board.piece_at(chess.parse_square(str(move)[:2])).symbol() == "Q":
            candidate_moves[find_best_queen_move(board.copy(), move)].append(move)
    return random.choice(sorted(candidate_moves.items(), key=lambda x: x[0], reverse=True)[0][1])

In [644]:
def make_best_black_move(board: chess.Board):
    king_distance = calculate_black_king_distance(board.copy())
    output = {}
    for move in board.legal_moves:
        board.push(move)
        output[calculate_black_king_distance(board)] = move
        board.pop()
    return output[min(output.keys())]


In [645]:
def show_board(board):
    clear_output()
    sleep(0.001)
    display(board)

def playing_loop(board, auto_play = True, show_autoplay = False):
    while board.is_game_over() == False:
        if len(board.move_stack) > 100:
            print("Draw by 50-move rule")
            break
        if board.turn == chess.WHITE:
            board.push(calculate_best_move(board))
        else:
            if auto_play:
                if show_autoplay:
                    show_board(board)
                    sleep(0.15)
                board.push(make_best_black_move(board))
                if show_autoplay:
                    show_board(board)
                    sleep(0.15)
            else:
                try:
                    show_board(board)
                    display(calculate_black_king_distance(board))
                    sleep(1)
                    player_input = input("Your move: ")
                    if player_input == "exit":
                        break
                    board.push_san(player_input)
                except Exception:
                    pass
    #show_board(board)

In [646]:
output = {}
for i in range(1000):
    board = chess.Board(fen="8/8/8/8/8/8/8/8 w - - 0 1")
    r = random.randint(0, 63)
    board.set_piece_at(r, chess.Piece(chess.QUEEN, chess.WHITE))
    while True:
        r = random.randint(0, 63)
        if board.piece_at(r) is None:
            board.set_piece_at(r, chess.Piece(chess.KING, chess.WHITE))
            break
    while True:
        r = random.randint(0, 63)
        if board.piece_at(r) is None and board.is_attacked_by(chess.WHITE, r) == False:
            board.set_piece_at(r, chess.Piece(chess.KING, chess.BLACK))
            break
    output[f"Game {i}"] = {"startring_board": board.fen()}
    playing_loop(board)#, show_autoplay=True)
    output[f"Game {i}"]["result"] = board.result()
    output[f"Game {i}"]["moves"] = board.move_stack
    output[f"Game {i}"]["game_length"] = len(board.move_stack)

In [647]:
print(Counter([output[i]["result"] for i in output]))
print(max([output[i]["game_length"] for i in output]))
print(max([output[i]["game_length"] for i in output]))

Counter({'1-0': 995, '1/2-1/2': 5})
57
1
